In [1]:
import numpy as np
import pandas as pd

# Data Preprocessing

This dataset is composed of multiple data sources, and we need to integrate them into a unified dataset through various methods for machine learning. Below is an introduction to these datasets.

## Datasets

### train_users.csv & test_users.csv

train_users: 213451 rows 16 colunms (23.70 MB)

test_users: 62096 rows 15 columns (6.44 MB)

| Column Name               | Description                                                                                                     |
|---------------------------|-----------------------------------------------------------------------------------------------------------------|
| **id**                   | User ID                                                                                                         |
| **date_account_created** | The date of account creation                                                                                    |
| **timestamp_first_active** | Timestamp of the first activity (can be earlier than account creation or first booking date)                  |
| **date_first_booking**   | Date of first booking                                                                                          |
| **gender**               | Gender                                                                                                         |
| **age**                  | Age                                                                                                            |
| **signup_method**        | Signup method                                                                                                |
| **signup_flow**          | The page a user came to sign up from                                                                           |
| **language**             | International language preference                                                                             |
| **affiliate_channel**    | The type of paid marketing channel                                                                             |
| **affiliate_provider**   | The marketing provider, e.g., Google, Craigslist, etc.                                                        |
| **first_affiliate_tracked** | The first marketing interaction before signing up                                                           |
| **signup_app**           | The app used to sign up                                                                                       |
| **first_device_type**    | The first type of device used                                                                                  |
| **first_browser**        | The first browser used                                                                                        |
| **country_destination**  | The target variable representing the country where the user booked                                             |

### sessions.csv

10567737 rows 6 colunms (602.38 MB)

| Column Name    | Description                                                                             |
|----------------|-----------------------------------------------------------------------------------------|
| **user_id**    | To be joined with the `id` column in the test users table                              |
| **action**     | The action performed by the user on the website                                        |
| **action_type**| The type of action                                                                     |
| **action_detail** | Details about the action                                                           |
| **device_type**| The type of device used for the action                                                 |
| **secs_elapsed** | The time elapsed (in seconds) while performing the action                           |

### countries.csv

| Column Name                    | Description                                                                                           |
|--------------------------------|-------------------------------------------------------------------------------------------------------|
| **country_destination**        | The destination country code                                                                          |
| **lat_destination**            | Latitude of the destination country                                                                 |
| **lng_destination**            | Longitude of the destination country                                                                |
| **distance_km**                | Distance in kilometers from the user's origin to the destination country                            |
| **destination_km2**            | Area of the destination country in square kilometers                                                |
| **destination_language**       | Primary language spoken in the destination country                                                  |
| **language_levenshtein_distance** | Levenshtein distance (edit distance) between the user's language preference and the destination language | 

### age_gender_bkts.csv

| Column Name                | Description                                                                                         |
|----------------------------|-----------------------------------------------------------------------------------------------------|
| **age_bucket**             | Age range or bucket (e.g., "20-24", "100+")                                                        |
| **country_destination**    | Destination country code                                                                           |
| **gender**                 | Gender (e.g., male, female)                                                                        |
| **population_in_thousands**| Population count for the specific age bucket and gender, in thousands                              |
| **year**                   | Year of the recorded population data                                                              |

## Preparation

First, we integrate all the data while processing both the training set and the test set.

In [2]:
train_set = pd.read_csv("data/train_users.csv")
test_set = pd.read_csv("data/test_users.csv")
test_set["country_destination"] = "NDF"
df = pd.concat([train_set, test_set], axis=0)

Some useful functions here.

In [3]:
def replace_with_nan(df, column_name, nan):
    df[column_name] = df[column_name].replace(nan, np.nan)


def split_date_column(df, column_name, date_format=None):
    if date_format:
        df[column_name] = pd.to_datetime(df[column_name], format=date_format)
    else:
        df[column_name] = pd.to_datetime(df[column_name])

    col_index = df.columns.get_loc(column_name)
    df.insert(col_index + 1, f"{column_name}_year", df[column_name].dt.year)
    df.insert(col_index + 2, f"{column_name}_month", df[column_name].dt.month)
    df.insert(col_index + 3, f"{column_name}_day", df[column_name].dt.day)
    df.insert(
        col_index + 4,
        f"{column_name}_yearmonth",
        (
            df[f"{column_name}_year"].astype(str)
            + df[f"{column_name}_month"].apply(lambda x: f"{x:02d}")
        ).astype(int),
    )
    df.insert(
        col_index + 5,
        f"{column_name}_week",
        (df[column_name] + pd.to_timedelta(3, unit="D")).dt.strftime("%U").astype(int),
    )
    df.insert(
        col_index + 6,
        f"{column_name}_yearmonthweek",
        (
            df[f"{column_name}_year"].astype(str)
            + df[f"{column_name}_month"].apply(lambda x: f"{x:02d}")
            + df[f"{column_name}_week"].apply(lambda x: f"{x:02d}")
        ).astype(int),
    )
    df.insert(
        col_index + 7,
        f"{column_name}_yearmonthday",
        (
            df[f"{column_name}_year"].astype(str)
            + df[f"{column_name}_month"].apply(lambda x: f"{x:02d}")
            + df[f"{column_name}_day"].apply(lambda x: f"{x:02d}")
        ).astype(int),
    )
    df.drop(columns=[column_name], inplace=True)
    return df


def count_nonzeros(column):
    column = column.fillna(0)
    column = np.where(column > 1, 1, column)
    return np.sum(column)

In [4]:
replace_with_nan(df, "gender", "-unknown-")
replace_with_nan(df, "first_affiliate_tracked", "untracked")
replace_with_nan(df, "first_device_type", "Other/Unknown")
replace_with_nan(df, "first_browser", "-unknown-")

## Time-Based Data

The variables such as the first activity, account creation, and first booking do not provide much valuable information individually. However, the relationships between them can offer significant insights. Therefore, we process these columns by calculating their differences to generate new columns.

At the same time, a full column of raw date data often mixes valuable information together, making it difficult for the model to capture. For example, disregarding the year, people generally take vacations during the summer months (May to August). However, if the year and month are combined, the model may struggle to identify this pattern. Therefore, we need to split such columns into separate components.

In [5]:
df["date_account_created_lag"] = (
    pd.to_datetime(df["date_account_created"])
    - pd.to_datetime(df["timestamp_first_active"], format="%Y%m%d%H%M%S")
).dt.days
df["date_first_booking_lag"] = (
    pd.to_datetime(df["date_first_booking"])
    - pd.to_datetime(df["date_account_created"])
).dt.days

df = split_date_column(df, "date_account_created")
df = split_date_column(df, "timestamp_first_active", date_format="%Y%m%d%H%M%S")
df.drop(columns=["date_first_booking"], inplace=True)

## Age

The **age_gender_bkts** dataset provides data based on age buckets, so we need to bucketize the original age data to merge it with this dataset.

Additionally, we notice that some ages are extraordinarily high but resemble recent years. Given the possibility that users mistakenly entered their birth year instead of age, we can perform error correction to retrieve more valid data. Of course, any data that remains unreasonable should also be excluded.

In [6]:
bins = list(np.arange(0, 100, 5)) + [np.int64(100), np.int64(200)]
labels = [f"{bins[i]}-{bins[i+1]-1}" for i in range(len(bins) - 2)] + [f"{bins[-2]}+"]

In [7]:
df["age"] = np.where(df["age"] >= 1915, 2015 - df["age"], df["age"])

col_index = df.columns.get_loc("age")
df.insert(
    col_index + 1,
    f"age_bucket",
    np.where((df["age"] < 13) | (df["age"] > 120), -1, df["age"]),
)

df["age_bucket"] = pd.cut(df["age_bucket"], bins=bins, labels=labels, right=False)

## Countries

In [8]:
countries = pd.read_csv("data/countries.csv")
countries

,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
0,AU,-26.853388,133.275160,15297.7440,7741220.0,eng,0.00
1,CA,62.393303,-96.818146,2828.1333,9984670.0,eng,0.00
2,DE,51.165707,10.452764,7879.5680,357022.0,deu,72.61
3,ES,39.896027,-2.487694,7730.7240,505370.0,spa,92.25
4,FR,46.232193,2.209667,7682.9450,643801.0,fra,92.06
5,GB,54.633220,-3.432277,6883.6590,243610.0,eng,0.00
6,IT,41.873990,12.564167,8636.6310,301340.0,ita,89.40
7,NL,52.133057,5.295250,7524.3203,41543.0,nld,63.22
8,PT,39.553444,-7.839319,7355.2534,92090.0,por,95.45
9,US,36.966427,-95.844030,0.0000,9826675.0,eng,0.00


In [9]:
df = pd.merge(
    df,
    countries[
        [
            "country_destination",
            "distance_km",
            "destination_km2",
            "language_levenshtein_distance",
        ]
    ].rename(
        columns={
            "distance_km": "destination_distance_km",
            "destination_km2": "destination_area",
            "language_levenshtein_distance": "destination_language_levenshtein_distance",
        }
    ),
    on="country_destination",
    how="left",
)

columns = [col for col in df.columns if col != "country_destination"] + [
    "country_destination"
]
df = df[columns]

## Population Pyramid

This dataset contains the population pyramid of different countries for the year 2015 (the only year available). The chosen integration method involves adding data on "the number of people in the same age group and gender in the destination country" to the original dataset through a series of operations.

This aligns with real-life experiences. For example, in terms of gender, countries like France, where females have a higher proportion, are often attractive destinations for women. Conversely, regions with strong patriarchal traditions, such as the Middle East and East Asia, may be less appealing or even raise safety concerns for women.

In [10]:
age_gender_bkts = pd.read_csv("data/age_gender_bkts.csv")
age_gender_bkts.drop(columns=["year"], inplace=True)
age_gender_bkts

,age_bucket,country_destination,gender,population_in_thousands
0,100+,AU,male,1.0
1,95-99,AU,male,9.0
2,90-94,AU,male,47.0
3,85-89,AU,male,118.0
4,80-84,AU,male,199.0
...,...,...,...,...
415,95-99,US,male,115.0
416,90-94,US,male,541.0
417,15-19,US,female,10570.0
418,85-89,US,male,1441.0


In [11]:
age_gender_bkts = age_gender_bkts.merge(
    countries[["country_destination", "destination_language"]],
    on="country_destination",
    how="left",
)

In [12]:
reshaped = []
for cl in age_gender_bkts["country_destination"].unique():
    subset = age_gender_bkts[age_gender_bkts["country_destination"] == cl]
    subset = subset.pivot_table(
        index=["destination_language", "age_bucket", "gender"],
        columns="country_destination",
        values="population_in_thousands",
        aggfunc="sum",
    ).reset_index()
    reshaped.append(subset)

age_gender_bkts_reshape = pd.concat(reshaped, ignore_index=True)

In [13]:
age_gender_bkts_grouped = (
    age_gender_bkts_reshape.groupby(["age_bucket", "gender", "destination_language"])
    .sum(numeric_only=True)
    .reset_index()
)
age_gender_bkts_grouped["language"] = age_gender_bkts_grouped[
    "destination_language"
].str[:2]
age_gender_bkts_grouped.drop(columns=["destination_language"], inplace=True)
age_gender_bkts_grouped["gender"] = age_gender_bkts_grouped["gender"].str.upper()
age_gender_bkts_grouped.to_feather("data/preprocessed/age_gender_bkts_grouped")
age_gender_bkts_grouped

country_destination,age_bucket,gender,AU,CA,DE,ES,FR,GB,IT,NL,PT,US,language
0,0-4,FEMALE,0.0,0.0,1713.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,de
1,0-4,FEMALE,781.0,991.0,0.0,0.0,0.0,1888.0,0.0,0.0,0.0,10306.0,en
2,0-4,FEMALE,0.0,0.0,0.0,0.0,1938.0,0.0,0.0,0.0,0.0,0.0,fr
3,0-4,FEMALE,0.0,0.0,0.0,0.0,0.0,0.0,1383.0,0.0,0.0,0.0,it
4,0-4,FEMALE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,438.0,0.0,0.0,nl
...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,95-99,MALE,0.0,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,fr
290,95-99,MALE,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0,0.0,0.0,it
291,95-99,MALE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,nl
292,95-99,MALE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,po


In [14]:
df = df.merge(
    age_gender_bkts_grouped,
    on=["age_bucket", "gender", "language"],
    how="left",
)
df.drop(columns=["age_bucket"], inplace=True)

## Session

The session data has already been preprocessed in another file; here, we will merge it into the dataset.

In [15]:
session = pd.read_feather("data/preprocessed/sessions_data")
session

feature,action_se_sum_10,action_se_sum_11,action_se_sum_12,action_se_sum_15,action_se_sum_about_us,action_se_sum_accept_decline,action_se_sum_account,action_se_sum_acculynk_bin_check_failed,action_se_sum_acculynk_bin_check_success,action_se_sum_acculynk_load_pin_pad,...,device_type_flg_sum_Chromebook,device_type_flg_sum_Linux Desktop,device_type_flg_sum_Mac Desktop,device_type_flg_sum_Opera Phone,device_type_flg_sum_Tablet,device_type_flg_sum_Windows Desktop,device_type_flg_sum_Windows Phone,device_type_flg_sum_iPad Tablet,device_type_flg_sum_iPhone,device_type_flg_sum_iPodtouch
id,,,,,,,,,,,,,,,,,,,,,
00023iyk9l,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,36.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN
0010k6l0om,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,63.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
001wyh0pz8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0028jgx1x1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
002qnbzfs5,301482.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,775.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzxox7jnrx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,89.0,NaN,NaN,NaN,NaN
zzy7t0y9cm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN
zzysuoqg6x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN


In [16]:
df = df.merge(session, how="left", on="id")

## Misc

In [17]:
df = df.astype(
    {
        "gender": "category",
        "signup_method": "category",
        "signup_flow": "category",
        "language": "category",
        "affiliate_channel": "category",
        "affiliate_provider": "category",
        "first_affiliate_tracked": "category",
        "signup_app": "category",
        "first_device_type": "category",
        "first_browser": "category",
        "country_destination": "category",
    }
)

Remove columns that are largely irrelevant or contain mostly invalid data.

In [18]:
nonzero_counts = df.drop(
    columns=["id", "country_destination"]
    + df.select_dtypes(include="category").columns.tolist()
).apply(count_nonzeros, axis=0)
low_frequency_cols = nonzero_counts[nonzero_counts <= 10].index
print("Columns to remove:", len(low_frequency_cols))

Columns to remove: 197


In [19]:
df = df.drop(columns=low_frequency_cols)

In [20]:
df.to_feather("data/preprocessed/all_data")
df

,id,date_account_created_year,date_account_created_month,date_account_created_day,date_account_created_yearmonth,date_account_created_week,date_account_created_yearmonthweek,date_account_created_yearmonthday,timestamp_first_active_year,timestamp_first_active_month,...,device_type_flg_sum_Blackberry,device_type_flg_sum_Chromebook,device_type_flg_sum_Linux Desktop,device_type_flg_sum_Mac Desktop,device_type_flg_sum_Tablet,device_type_flg_sum_Windows Desktop,device_type_flg_sum_Windows Phone,device_type_flg_sum_iPad Tablet,device_type_flg_sum_iPhone,device_type_flg_sum_iPodtouch
0,gxn3p5htnn,2010,6,28,201006,26,20100626,20100628,2009,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,820tgsjxq7,2011,5,25,201105,21,20110521,20110525,2009,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4ft3gnwmtx,2010,9,28,201009,39,20100939,20100928,2009,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,bjjt8pjhuk,2011,12,5,201112,49,20111249,20111205,2009,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,87mebub9p4,2010,9,14,201009,37,20100937,20100914,2009,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275542,cv0na2lf5a,2014,9,30,201409,39,20140939,20140930,2014,9,...,NaN,NaN,NaN,NaN,NaN,89.0,NaN,NaN,4.0,NaN
275543,zp8xfonng8,2014,9,30,201409,39,20140939,20140930,2014,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
275544,fa6260ziny,2014,9,30,201409,39,20140939,20140930,2014,9,...,NaN,NaN,NaN,NaN,NaN,78.0,NaN,NaN,NaN,NaN
275545,87k0fy4ugm,2014,9,30,201409,39,20140939,20140930,2014,9,...,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,4.0,NaN
